In [ ]:
# basic dataframe and operations
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# manipulation and preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel

# models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

# measuring results
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, mean_absolute_error, roc_auc_score, precision_score, recall_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

# warning suppression
import warnings
from sklearn.exceptions import ConvergenceWarning

In [ ]:
    # Import the csv training dataset to a pandas dataframe
# The dataset is expected in the same directory as this notebook
#   under a subfolder path datasets/
data_raw_input = pd.read_csv('datasets/train.csv')
# Show the shape of the dataset
data_raw_input.shape


In [ ]:
# Display a few rows from the training data
data_raw_input.head(20)

In [ ]:
# Get the count of nulls per column
# Turns out we don't have any
print("Nulls:")
print(data_raw_input.isnull().sum().sum())
print("Na count:")
print(data_raw_input.isna().sum().sum())
print("Duplicate rows:")
print(data_raw_input.duplicated(keep='first').sum())